# FCA analysis - preprocessing

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import os
import sys


def MovePlots(plotpattern, subplotdir):
    os.system('mkdir -p '+str(sc.settings.figdir)+'/'+subplotdir)
    os.system('mv '+str(sc.settings.figdir)+'/*'+plotpattern+'** '+str(sc.settings.figdir)+'/'+subplotdir)


sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = './figures/preprocessing/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable

/home/jovyan/my-conda-envs/anndata06/lib/python3.9/site-packages/anndata/core/anndata.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


scanpy==1.4.4 anndata==0.6.22 umap==0.5.1 numpy==1.20.0 scipy==1.6.2 pandas==1.2.1 scikit-learn==0.24.1 statsmodels==0.12.2


'/home/jovyan/my-conda-envs/anndata06/bin/python'

## Import data
Import sample metadata

In [2]:
data_dir = '/nfs/users/nfs_l/lg18/team292/lg18/gonads/data/scRNAseq/FCA/'
meta = pd.read_csv(data_dir+'rawdata/meta_noimmune_v2.csv',index_col=0)
meta['stage'] = meta['stage'].astype('str')
plotmeta = list(meta.columns)
plotmeta.append('sample')
print('Number of samples: ', meta.index.size)

Number of samples:  53


In [3]:
meta

,individual,location,stage,sex,study,batch_collection,process,cryopreserved,TP,5v1.1
sample,,,,,,,,,,
FCA_GND8047885,F81,Gonad,8.8,female,FCA,A,CD45-,No,U,5v1.1
FCA_GND8125923,F86,Gonad,7.5,female,FCA,B,CD45-,No,U,5v1.1
FCA_GND8125925,F87,Gonad,11.0,female,FCA,B,CD45-,No,U,5v1.1
FCA_GND8103050,F83,Gonad,17.0,female,FCA,C,CD45-,No,U,5v1.1
FCA_GND8103053,F84,Gonad,9.0,female,FCA,Cpoor,CD45-,No,U,5v1.1
FCA_GND8622625,F89,Gonad,8.4,female,FCA,E,total,No,U,5v1.1
FCA_GND8622626,F89,Gonad,8.4,female,FCA,E,total,No,U,5v1.1
FCA_GND8622628,F91,Gonad,11.0,female,FCA,E,CD45-,No,U,5v1.1
FCA_GND8715408,F96,Gonad,14.0,female,FCA,F,CD45-,No,Med,5v1.1


## Preprocessing
### Load 10x 

Filter: 1) cells (< 10 genes); 2) genes (< 3 cells) 

Quantify: 1) % mitochondrial genes; 2) total counts


In [4]:
holder = []
for sample in meta.index:
    print(sample)
    # Load 10x data as AnnData
    holder.append(sc.read_10x_mtx(data_dir+'rawdata/'+sample+'/filtered_feature_bc_matrix/',cache=True)) 
    # Set names of observation as sample + _ + barcode/probe
    holder[-1].obs_names = [sample+'_'+i.split('-')[0] for i in holder[-1].obs_names]
    # Filer genes expressed in less than 3 cells
    sc.pp.filter_genes(holder[-1], min_cells=3)
    # Filer cells with less than 10 genes expressed
    sc.pp.filter_cells(holder[-1], min_genes=150)
    # add in metadata
    holder[-1].obs['sample'] = sample
    for val in meta.columns:
        holder[-1].obs[val] = meta[val][sample]
    # Extract mitochondial genes
    mito_genes = [name for name in holder[-1].var_names if name.startswith('MT-')]
    #for each cell compute fraction of counts in mito genes vs. all genes
    #the `.A1` is only necessary, as X is sparse - it transform to a dense array after summing
    holder[-1].obs['percent_mito'] = np.sum(
        holder[-1][:, mito_genes].X, axis=1).A1 / np.sum(holder[-1].X, axis=1).A1
    #add the total counts per cell as observations-annotation to adata
    holder[-1].obs['n_counts'] = holder[-1].X.sum(axis=1).A1
    print('Total number of cells: {:d}'.format(holder[-1].n_obs))
    print('Total number of genes: {:d}'.format(holder[-1].n_vars))

FCA_GND8047885
... reading from cache file cache/nfs-users-nfs_l-lg18-team292-lg18-gonads-data-scRNAseq-FCA-rawdata-FCA_GND8047885-filtered_feature_bc_matrix-matrix.h5ad


/home/jovyan/my-conda-envs/anndata06/lib/python3.9/site-packages/anndata/core/anndata.py:1237: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(df_full[k]):


filtered out 13510 genes that are detectedin less than 3 cells


Trying to set attribute `.var` of view, making a copy.


filtered out 15 cells that haveless than 150 genes expressed
Total number of cells: 9227
Total number of genes: 20028
FCA_GND8125923
... reading from cache file cache/nfs-users-nfs_l-lg18-team292-lg18-gonads-data-scRNAseq-FCA-rawdata-FCA_GND8125923-filtered_feature_bc_matrix-matrix.h5ad
filtered out 12810 genes that are detectedin less than 3 cells


...

filtered out 49 cells that haveless than 150 genes expressed
Total number of cells: 8046
Total number of genes: 23592
HD_F_GON9883859
... reading from cache file cache/nfs-users-nfs_l-lg18-team292-lg18-gonads-data-scRNAseq-FCA-rawdata-HD_F_GON9883859-filtered_feature_bc_matrix-matrix.h5ad
filtered out 14469 genes that are detectedin less than 3 cells


Trying to set attribute `.var` of view, making a copy.


filtered out 67 cells that haveless than 150 genes expressed
Total number of cells: 4562
Total number of genes: 19069
HD_F_GON9883868
... reading from cache file cache/nfs-users-nfs_l-lg18-team292-lg18-gonads-data-scRNAseq-FCA-rawdata-HD_F_GON9883868-filtered_feature_bc_matrix-matrix.h5ad
filtered out 12408 genes that are detectedin less than 3 cells


Trying to set attribute `.var` of view, making a copy.


filtered out 9 cells that haveless than 150 genes expressed
Total number of cells: 5216
Total number of genes: 21130


In [ ]:
# confirm N samples
print(len(holder))
# merge datasets
adata = holder[0].concatenate(holder[1:],join='outer',index_unique=None)
# copy of this matrix in Compressed Sparse Row format
adata.X = adata.X.tocsr()
adata

53


### QC pplots

Plot distributions of the values n_genes, n_counts and percent_mito

In [ ]:
print('Total number of cells: {:d}'.format(adata.n_obs))
print('Total number of genes: {:d}'.format(adata.n_vars))

### Filter cells with few genes
Check number of genes per cell distribution and filter cells accordingly

In [ ]:
plt.hist(adata.obs['n_genes'], bins = 100)
plt.axvline(200, linestyle = '--', color = 'red')

In [ ]:
sc.pp.filter_cells(adata, min_genes=300)

In [ ]:
print('Total number of cells: {:d}'.format(adata.n_obs))
print('Total number of genes: {:d}'.format(adata.n_vars))

### Filter genes expressed in less than 3 cells

In [ ]:
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
print('Total number of cells: {:d}'.format(adata.n_obs))
print('Total number of genes: {:d}'.format(adata.n_vars))


### Filter cells with large % mitochondrial genes

In [ ]:
plt.hist(adata.obs['percent_mito'], bins = 100, cumulative=True)
plt.axvline(0.1, linestyle = '--', color = 'red')
plt.axvline(0.2, linestyle = '--', color = 'darkred')
plt.axhline(adata.n_obs*0.99, linestyle = '-', color = 'green')

In [ ]:
# >20%
adata = adata[adata.obs['percent_mito'] < 0.2, :]

In [ ]:
print('Total number of cells: {:d}'.format(adata.n_obs))
print('Total number of genes: {:d}'.format(adata.n_vars))

# Add metadata 

In [ ]:
# format some metadata
# PCW as numerical
adata.obs['PCW'] = adata.obs.stage.astype('float').tolist()
# source: L=london; N=newcastle
adata.obs['sample_source'] = 'N'
idx = np.array(['Hrv' in i for i in adata.obs['individual']])
adata.obs.at[ idx, 'sample_source']  = 'L'

In [ ]:
# Add cluster name and doublet information
clu_annot = pd.read_csv('figures_manual_annotation/clustering_metadata.csv', header=0, index_col=0)
import collections 

if collections.Counter(adata.obs.index) == collections.Counter(clu_annot.index): 
    print ("The lists are identical") 
else : 
    print ("The lists are not identical") 
    
vars2import = ['clusters_manual','old_clusters','louvain', 'is_doublet', 'scrublet_cluster_score', 'scrublet_score']
for var in vars2import:
    adata.obs[var] = clu_annot[var]

In [ ]:
adata.obs.head()

# Save

In [ ]:
# del adata.obs['location']
del adata.obs['clusters']

In [ ]:
adata.write('/nfs/team292/lg18/with_valentina/FCA-M5-annotatedCluster4Seurat.h5ad')